In [1]:
from libs.GPT import GPT
from libs.CorpusDataset import CorpusDataset
import torch
import torch.nn as nn
from transformers import AutoTokenizer
import glob
torch.set_default_device('cuda')
tokenizer = AutoTokenizer.from_pretrained("ikit-claw-nlp/toy-llm")
GPT_CONFIG_124M = {
    "vocab_size": tokenizer.vocab_size,
    "pad_idx": tokenizer.convert_tokens_to_ids("<pad>"),
    "context_length": 256, #max context length
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}


In [2]:
model = GPT(GPT_CONFIG_124M)
model = nn.DataParallel(model)

In [3]:
text_files = glob.glob("data/text/*.txt")
train_text_files = text_files[0: int(len(text_files)*0.8)]
valid_text_files = text_files[int(len(text_files)*0.8): int(len(text_files)*0.9)]
test_text_files = text_files[int(len(text_files)*0.9):]

train_dataset = CorpusDataset(train_text_files, window_size=GPT_CONFIG_124M['context_length'], step_length=1, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=8)

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 4e-4,
    weight_decay=0.1
)
max_epoch_num = 5
for _ in range(max_epoch_num):
    for idx, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        logits = model(x)
        logits = logits.flatten(0, 1)
        y = y.flatten(0,1)
        loss = torch.nn.functional.cross_entropy(logits, y)
        if idx % 100 == 0:
            print("Batch", idx, "Loss", loss)
            print()
        loss.backward()
        optimizer.step()

        print("Training Batch", idx, end="\r")

Loading the dataset data/text/article_1-1000.txt into memory...
Converting the dataset to token ids...
Conversion Complete. torch.Size([7540024]) Tokens in the corpus.
Batch 0 Loss tensor(10.9617, device='cuda:0', grad_fn=<NllLossBackward0>)



TypeError: 'ends' is an invalid keyword argument for print()

In [ ]:
logits = model(x)
print(logits.flatten(0,1).shape)
print(y.flatten(0,1).shape)
loss = torch.nn.functional.cross_entropy(logits.flatten(0,1), y.flatten(0,1))